# Veta Basics

Veta is comprised of three main components (called classes in computer science): the item, respondent, and survey. Each of these classes is designed around how an LEAS survey would typically be conducted with the idea being that the programmer should be able to simply and naturally score LEAS data. Additionally, veta implements a number of scoring methodologies that can be applied across entire surveys or just to responses of responents aswell as several basic analysis functions which can be applied to visualize survey data.

### The Item Class

The item class is used to hold the information corresponding to a single LEAS question response. For example, if the question was "How would you feel if you received a poor grade on a test? How would the instructor feel?" and the respondent answered, "I would feel bad, and I don't think that the instructor would care.", you could represent this in veta as follows:


In [6]:
#Adding the veta directory to the environment path
import sys
sys.path.insert(0,'../')

#Import the Item class
from item import Item

#Creates a new item. By default, the sentence is placed in the 'Self' category if not broken up manually
new_item = Item("I would feel bad, and I don't think that the instructor would care.")
print(new_item)

#Creates the same item but, the sentence is broken up manually into self and other components
new_item = Item("I would feel bad, and", "I don't think that the instructor would care.")
print(new_item)

Self: i would feel bad  and i don't think that the instructor would care 
Other: 

Self: i would feel bad  and
Other: i don't think that the instructor would care 



The item class requires you to manually break up the response into components that relate to the 'self' (i.e. I would feel bad) and components of the response which correspond to another party (i.e. they would feel good). This distinction is used for a subset of the availiable scoring modules and is not strictly required. If you do not plan to use those scoring modules (e.g. 3345), then just ignore this distinction. 

Once an item is created, you can score the item using scoring module (see Scoring) and add additional information that are related specifically to that response (e.g. time taken).

In [7]:
new_item.add_additional_info("time taken (sec)", 85)
print(new_item)

Self: i would feel bad  and
Other: i don't think that the instructor would care 
time taken (sec): 85



### The Respondent Class

The respondent class holds all information corresponding to one respondent of an LEAS survey. This includes all of the LEAS items (question responses) as well as meta data about the respondent (e.g. age, gender, education). The following code initializes a respondent and highlights some basic functionality of the class.

In [8]:
#Import the Item class
from respondent import Respondent

#Initialize a new respondent
new_respondent = Respondent()
#Each respondent you create has a unique ID based on the order they were created
print(f"This is how you can get the respondent id: {new_respondent.id}\n")

#You can add items without initializing them yourself
new_respondent.add_item("this is a test item", "hello")

#Or you can add existing items
new_respondent.add_item(new_item)

#At any time you can print the respondent and it will show you all of the item information
print(new_respondent)

#Add some meta data about the respondent. Must be numeric data
new_respondent.add_additional_info("Age", 30)
new_respondent.add_additional_info("Gender", 1)
new_respondent.add_additional_info("Education", 5)

#To get all of the data, including total scores, you can convert the respondent info to an array
#Each row corresponds to an item (the last row is cummulative/per_respondent information)
#Each column corresponds to a type of information, in this example, they correspond to Age, Gender, and Education
#After scoring, there will be additional columns with the per item and cummulative scores.
new_resp_array = new_respondent.to_array()

#When you convert to an array, you can access which columns are which with the col_names parameter.
print(new_respondent.col_names)
print(new_resp_array)


This is how you can get the respondent id: 1

Respondent ID 1:

Item 1:
Self: this is a test item
Other: hello

Item 2:
Self: i would feel bad  and
Other: i don't think that the instructor would care 
time taken (sec): 85
['Age', 'Education', 'Gender']
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [30.  5.  1.]]


### The Survey Class

The survey class is used to hold all of the information which pertains to multiple LEAS respondents. Survey data can be read from Excel files for ease of use. The following demonstrates how to initialize a survey class manually, or read in the data from an Excel file.

In [9]:
#Import the Item class
from survey import Survey

#Initialize the Survey Object
new_survey = Survey()

#Add some respondents to the survey
respondent1 = Respondent()
respondent1.add_item("First respondent First Item")
respondent1.add_item("First respondent Second Item")
respondent1.add_additional_info("Age", 15)
respondent1.add_additional_info("Gender", 0)
respondent1.add_additional_info("Education", 3)

respondent2 = Respondent()
respondent2.add_item("Second respondent First Item")
respondent2.add_item("Second respondent Second Item")
respondent2.add_additional_info("Age", 57)
respondent2.add_additional_info("Gender", 1)
respondent2.add_additional_info("Education", 2)

new_survey.add_respondent(respondent1)
new_survey.add_respondent(respondent2)
new_survey.add_respondent(new_respondent)

#Display the survey info
print(new_survey)


Respondent ID 2:

Item 1:
Self: first respondent first item
Other: 

Item 2:
Self: first respondent second item
Other:

Respondent ID 3:

Item 1:
Self: second respondent first item
Other: 

Item 2:
Self: second respondent second item
Other:

Respondent ID 1:

Item 1:
Self: this is a test item
Other: hello

Item 2:
Self: i would feel bad  and
Other: i don't think that the instructor would care 
time taken (sec): 85


In [10]:
#You can also compute a summary of all of the information in the survey, this will be more relevant when we look at scoring
#It is important to note that if some of your repondents are missing information, it will not show up in the summary which
#can be confusing during analysis
new_survey.compute_summary()
print(new_survey.summary)

{'Age': array([15., 57., 30.]), 'Gender': array([0., 1., 1.]), 'Education': array([3., 2., 5.])}


### Wordlists

One of the most fundamental components of veta is actually scoring the LEAS items and respondents. Veta comes with a number of standard LEAS scoring methodologies as well as some non-standard methodologies that we came up with. The scoring modules are implemented as separate child classes of the ScoringModule class. Most of the scoring modules require an LEAS wordlist in order to work properly. Currently, wordlists must be read from a excel file with appropriate formatting (see wordlist.xlsx as an example). The following example shows how to initialize a Wordlist object and add it to a survey.

In [11]:
#Import the wordlist object 
from wordlist import Wordlist

#Initialize the wordlist object
new_wordlist = Wordlist("wordlist.xlsx")

#Adds the wordlist to all respondents/items in the survey
#You can also add a wordlist to just a single item/respondent in an analogous way
new_survey.add_wordlist(new_wordlist)

#The wordlist got added recursively to the 'new_item' since it is part of the survey 
print(new_item.wordlist)

Angst:               3
enttäuscht:          3
erleichtert:         3
glücklich :          3
gut:                 2
Kopfschmerzen:       1
Mitleid:             3
schlecht:            2
Schmerz:             1
schuldig:            3
traurig:             3
weinen:              2
weint:               2
weinte:              2
will:                3
wollen:              3
wollt:               3
Wut:                 3
wütend:              3
zittern:             1



### Scoring, and Scoring Modules

The item, respondent, and survey classes all have the ability to be scored. Scoring a survey will score all of its respodents, and scoring a respondent will subsequently score all of its items. The easiest way to score an LEAS questionaire is to properly define a survey and then run the survey.score function for the desired modules. You can also individually score respondents and items.

In [12]:
#Import the desired scoring methods
from scoring_modules.allsum import allsum
from scoring_modules.highestN import highestN

#create an example item containing words from the example German wordlist
example_item = Item("gut will wut zittern test test test")

#Add the example German Wordlist
example_item.add_wordlist(new_wordlist)

#Score the item using allsum, this will add the scores of the words 
# gut, will, wut, and zittern (2 + 3 + 3 + 1 = 9)
example_item.score(allsum())

#Score the item using highestN (N = 2), this will add the two highest scoring 
# words from the wordlist (will and wut) for a score of 3 + 3 = 6
example_item.score(highestN(2))
print(example_item)

#You can put all of the scoring modules in a list and run them on an entire survey
modules = [allsum(), highestN(4)]
#This will recursively score all of the respondents and items in the entire survey with the allsum and highest4 modules
new_survey.score(*modules)
#Computes a summary of all of the scores and auxilary information
new_survey.compute_summary()
print(new_survey.summary)


Self: gut will wut zittern test test test
Other: 
allsum: 9
highestN: 6

{'Age': array([15., 57., 30.]), 'Gender': array([0., 1., 1.]), 'Education': array([3., 2., 5.]), 'allsum': array([0., 0., 0.]), 'highestN': array([0., 0., 0.])}
